# 第一步：导入必要的库

In [1]:
import pandas as pd
import numpy as np
import math
from collections import Counter

# 第二步：创建数据集

In [5]:
# 修正数据集
data = {
    '年龄': ['青年', '青年', '青年', '青年', '青年', '中年', '中年', '中年', '中年', '中年', '老年', '老年', '老年', '老年', '老年'],
    '有工作': ['否', '否', '是', '是', '否', '否', '否', '是', '否', '否', '否', '否', '是', '是', '否'],
    '有自己的房子': ['否', '否', '否', '是', '否', '否', '否', '是', '是', '是', '是', '是', '否', '否', '否'],
    '信贷情况': ['一般', '好', '好', '一般', '一般', '一般', '好', '好', '非常好', '非常好', '非常好', '好', '好', '非常好', '一般'],
    '类别': ['否', '否', '是', '是', '否', '否', '否', '是', '是', '是', '是', '是', '是', '是', '否']
}

df = pd.DataFrame(data)

# 设置pandas显示选项，确保中文正确对齐
pd.set_option('display.unicode.east_asian_width', True)
pd.set_option('display.width', 100)

print("修正后的原始数据集：")
print(df.to_string(index=True))  # 使用to_string确保格式正确
print(f"\n数据集信息:")
print(f"- 总样本数: {len(df)}")
print(f"- 特征数: {len(df.columns)-1}")
print(f"- 类别分布: 是={sum(df['类别']=='是')}, 否={sum(df['类别']=='否')}")

修正后的原始数据集：
    年龄 有工作 有自己的房子 信贷情况 类别
0   青年     否           否     一般   否
1   青年     否           否       好   否
2   青年     是           否       好   是
3   青年     是           是     一般   是
4   青年     否           否     一般   否
5   中年     否           否     一般   否
6   中年     否           否       好   否
7   中年     是           是       好   是
8   中年     否           是   非常好   是
9   中年     否           是   非常好   是
10  老年     否           是   非常好   是
11  老年     否           是       好   是
12  老年     是           否       好   是
13  老年     是           否   非常好   是
14  老年     否           否     一般   否

数据集信息:
- 总样本数: 15
- 特征数: 4
- 类别分布: 是=9, 否=6


# 第三步：计算熵的函数

In [6]:
def calculate_entropy(labels):
    """
    计算数据集的熵
    labels: 类别标签列表
    """
    total_count = len(labels)
    if total_count == 0:
        return 0
    
    # 统计每个类别的数量
    label_counts = Counter(labels)
    entropy = 0.0
    
    for count in label_counts.values():
        probability = count / total_count
        entropy -= probability * math.log2(probability)
    
    return entropy

# 测试熵的计算
test_labels = df['类别'].tolist()
print(f"整个数据集的熵: {calculate_entropy(test_labels):.4f}")

整个数据集的熵: 0.9710


# 第四步：计算信息增益

In [7]:
def calculate_information_gain(data, feature_name, target_name='类别'):
    """
    计算某个特征的信息增益
    data: 数据集
    feature_name: 特征名称
    target_name: 目标变量名称
    """
    # 计算原始熵
    total_entropy = calculate_entropy(data[target_name])
    
    # 根据特征值分割数据
    feature_values = data[feature_name].unique()
    weighted_entropy = 0.0
    total_count = len(data)
    
    for value in feature_values:
        subset = data[data[feature_name] == value]
        subset_entropy = calculate_entropy(subset[target_name])
        subset_weight = len(subset) / total_count
        weighted_entropy += subset_weight * subset_entropy
    
    # 信息增益 = 原始熵 - 加权后的熵
    information_gain = total_entropy - weighted_entropy
    return information_gain

# 测试信息增益计算
features = ['年龄', '有工作', '有自己的房子', '信贷情况']
print("各特征的信息增益：")
for feature in features:
    ig = calculate_information_gain(df, feature)
    print(f"{feature}: {ig:.4f}")

各特征的信息增益：
年龄: 0.0830
有工作: 0.3237
有自己的房子: 0.4200
信贷情况: 0.3630


# 第五步：选择最佳分割特征

In [8]:
def choose_best_feature(data, features, target_name='类别'):
    """
    选择信息增益最大的特征
    """
    best_feature = None
    best_gain = -1
    
    for feature in features:
        gain = calculate_information_gain(data, feature, target_name)
        print(f"特征 '{feature}' 的信息增益: {gain:.4f}")
        
        if gain > best_gain:
            best_gain = gain
            best_feature = feature
    
    return best_feature, best_gain

# 测试选择最佳特征
available_features = ['年龄', '有工作', '有自己的房子', '信贷情况']
best_feature, best_gain = choose_best_feature(df, available_features)
print(f"\n最佳分割特征: '{best_feature}', 信息增益: {best_gain:.4f}")

特征 '年龄' 的信息增益: 0.0830
特征 '有工作' 的信息增益: 0.3237
特征 '有自己的房子' 的信息增益: 0.4200
特征 '信贷情况' 的信息增益: 0.3630

最佳分割特征: '有自己的房子', 信息增益: 0.4200


# 第六步：构建决策树节点类

In [10]:
class TreeNode:
    """决策树节点类"""
    def __init__(self, feature_name=None, feature_value=None, is_leaf=False, label=None):
        self.feature_name = feature_name  # 分裂特征名称
        self.feature_value = feature_value  # 到达此节点的特征值
        self.is_leaf = is_leaf  # 是否为叶节点
        self.label = label  # 叶节点的类别标签
        self.children = {}  # 子节点字典 {特征值: 节点}
    
    def add_child(self, feature_value, child_node):
        self.children[feature_value] = child_node
    
    def __repr__(self):
        if self.is_leaf:
            return f"LeafNode(类别={self.label})"
        else:
            return f"TreeNode(特征={self.feature_name}, 子节点数={len(self.children)})"

# 第七步：递归构建决策树

In [11]:
def build_decision_tree(data, features, target_name='类别', parent_feature_value=None):
    """
    递归构建决策树
    """
    # 创建当前节点
    current_node = TreeNode(feature_value=parent_feature_value)
    
    # 情况1: 如果所有样本都属于同一类别，则设为叶节点
    labels = data[target_name].tolist()
    if len(set(labels)) == 1:
        current_node.is_leaf = True
        current_node.label = labels[0]
        return current_node
    
    # 情况2: 如果没有特征可用于分割，则选择多数类别
    if len(features) == 0:
        current_node.is_leaf = True
        current_node.label = Counter(labels).most_common(1)[0][0]
        return current_node
    
    # 选择最佳分割特征
    best_feature, best_gain = choose_best_feature(data, features, target_name)
    print(f"选择特征 '{best_feature}' 进行分割，信息增益: {best_gain:.4f}")
    
    current_node.feature_name = best_feature
    
    # 从剩余特征中移除当前选择的最佳特征
    remaining_features = [f for f in features if f != best_feature]
    
    # 对最佳特征的每个值创建分支
    for feature_value in data[best_feature].unique():
        # 获取该特征值对应的子集
        subset = data[data[best_feature] == feature_value]
        
        if len(subset) == 0:
            # 如果子集为空，创建叶节点，选择父节点中多数的类别
            leaf_node = TreeNode(is_leaf=True, 
                               label=Counter(labels).most_common(1)[0][0],
                               feature_value=feature_value)
            current_node.add_child(feature_value, leaf_node)
        else:
            # 递归构建子树
            child_node = build_decision_tree(subset, remaining_features, target_name, feature_value)
            current_node.add_child(feature_value, child_node)
    
    return current_node

# 构建决策树
print("开始构建决策树...")
decision_tree = build_decision_tree(df, available_features)
print("\n决策树构建完成！")

开始构建决策树...
特征 '年龄' 的信息增益: 0.0830
特征 '有工作' 的信息增益: 0.3237
特征 '有自己的房子' 的信息增益: 0.4200
特征 '信贷情况' 的信息增益: 0.3630
选择特征 '有自己的房子' 进行分割，信息增益: 0.4200
特征 '年龄' 的信息增益: 0.2516
特征 '有工作' 的信息增益: 0.9183
特征 '信贷情况' 的信息增益: 0.4739
选择特征 '有工作' 进行分割，信息增益: 0.9183

决策树构建完成！


# 第八步：预测函数

In [12]:
def predict(tree, sample):
    """
    使用决策树预测样本类别
    """
    # 如果是叶节点，直接返回类别
    if tree.is_leaf:
        return tree.label
    
    # 获取样本在分裂特征上的值
    feature_value = sample[tree.feature_name]
    
    # 如果该特征值在决策树中有对应的分支，则继续预测
    if feature_value in tree.children:
        return predict(tree.children[feature_value], sample)
    else:
        # 如果遇到未知的特征值，返回None
        return None

# 测试预测
print("\n测试预测：")
test_sample = {
    '年龄': '青年',
    '有工作': '否', 
    '有自己的房子': '否',
    '信贷情况': '一般'
}

prediction = predict(decision_tree, test_sample)
print(f"测试样本预测结果: {prediction}")


测试预测：
测试样本预测结果: 否


# 第九步：可视化决策树

In [13]:
def print_tree(node, indent=""):
    """以文本形式打印决策树"""
    if node.is_leaf:
        print(f"{indent}类别: {node.label}")
    else:
        print(f"{indent}{node.feature_name}?")
        for feature_value, child_node in node.children.items():
            print(f"{indent}  {feature_value} -> ", end="")
            if child_node.is_leaf:
                print(f"类别: {child_node.label}")
            else:
                print()
                print_tree(child_node, indent + "    ")

print("\n构建的决策树结构：")
print_tree(decision_tree)


构建的决策树结构：
有自己的房子?
  否 -> 
    有工作?
      否 -> 类别: 否
      是 -> 类别: 是
  是 -> 类别: 是


In [17]:
def print_tree_enhanced(node, indent="", prefix="", is_last=True):
    """
    增强版树形可视化，使用Unicode字符创建更美观的树形结构
    """
    # 当前节点的连接符
    if prefix:
        print(f"{indent}{prefix}", end="")
    
    if node.is_leaf:
        print(f"📄 类别: {node.label}")
    else:
        print(f"❓ {node.feature_name}?")
    
    # 处理子节点
    if not node.is_leaf:
        children_items = list(node.children.items())
        for i, (feature_value, child_node) in enumerate(children_items):
            # 判断是否是最后一个子节点
            last_child = (i == len(children_items) - 1)
            
            # 确定连接符
            if prefix:
                new_prefix = "    " if is_last else "│   "
            else:
                new_prefix = ""
            
            # 子节点的连接线
            connector = "└── " if last_child else "├── "
            
            # 打印特征值分支
            print(f"{indent}{new_prefix}{connector}{feature_value}", end="")
            
            # 递归打印子树
            new_indent = indent + new_prefix
            if last_child:
                new_indent += "    "
            else:
                new_indent += "│   "
                
            print_tree_enhanced(child_node, new_indent, "", last_child)

def visualize_complete_tree():
    print("\n" + "="*50)
    print("决策树可视化")
    print("="*50)
    
    # 构建完整的决策树
    features = ['年龄', '有工作', '有自己的房子', '信贷情况']
    tree = build_decision_tree(df, features)
    
    print("文本树形结构:")
    print("-" * 30)
    print_tree(tree)
    
    print("\n增强版树形结构:")
    print("-" * 30)
    print_tree_enhanced(tree)
    
    return tree

# 运行可视化
final_tree = visualize_complete_tree()


决策树可视化
各特征的信息增益:
------------------------------
  年龄         : 0.0830
  有工作        : 0.3237
  有自己的房子     : 0.4200
  信贷情况       : 0.3630
深度0: 选择特征 '有自己的房子'
各特征的信息增益:
------------------------------
  年龄         : 0.2516
  有工作        : 0.9183
  信贷情况       : 0.4739
  深度1: 选择特征 '有工作'
文本树形结构:
------------------------------
有自己的房子?
 ├── 否
 │   有工作?
 │    ├── 否 → 类别: 否
 │    └── 是 → 类别: 是
 └── 是 → 类别: 是

增强版树形结构:
------------------------------
❓ 有自己的房子?
├── 否❓ 有工作?
│   ├── 否📄 类别: 否
│   └── 是📄 类别: 是
└── 是📄 类别: 是


# 第十步：完整测试

In [14]:
# 测试所有训练样本
print("\n训练集预测结果：")
correct = 0
total = len(df)

for i, row in df.iterrows():
    prediction = predict(decision_tree, row)
    actual = row['类别']
    status = "✓" if prediction == actual else "✗"
    if prediction == actual:
        correct += 1
    print(f"样本{i+1}: 预测={prediction}, 实际={actual} {status}")

accuracy = correct / total * 100
print(f"\n训练集准确率: {accuracy:.2f}%")


训练集预测结果：
样本1: 预测=否, 实际=否 ✓
样本2: 预测=否, 实际=否 ✓
样本3: 预测=是, 实际=是 ✓
样本4: 预测=是, 实际=是 ✓
样本5: 预测=否, 实际=否 ✓
样本6: 预测=否, 实际=否 ✓
样本7: 预测=否, 实际=否 ✓
样本8: 预测=是, 实际=是 ✓
样本9: 预测=是, 实际=是 ✓
样本10: 预测=是, 实际=是 ✓
样本11: 预测=是, 实际=是 ✓
样本12: 预测=是, 实际=是 ✓
样本13: 预测=是, 实际=是 ✓
样本14: 预测=是, 实际=是 ✓
样本15: 预测=否, 实际=否 ✓

训练集准确率: 100.00%


# Update

In [15]:
import pandas as pd
import numpy as np
import math
from collections import Counter

# 修正数据集
data = {
    '年龄': ['青年', '青年', '青年', '青年', '青年', '中年', '中年', '中年', '中年', '中年', '老年', '老年', '老年', '老年', '老年'],
    '有工作': ['否', '否', '是', '是', '否', '否', '否', '是', '否', '否', '否', '否', '是', '是', '否'],
    '有自己的房子': ['否', '否', '否', '是', '否', '否', '否', '是', '是', '是', '是', '是', '否', '否', '否'],
    '信贷情况': ['一般', '好', '好', '一般', '一般', '一般', '好', '好', '非常好', '非常好', '非常好', '好', '好', '非常好', '一般'],
    '类别': ['否', '否', '是', '是', '否', '否', '否', '是', '是', '是', '是', '是', '是', '是', '否']
}

df = pd.DataFrame(data)

# 设置pandas显示选项，确保中文正确对齐
pd.set_option('display.unicode.east_asian_width', True)
pd.set_option('display.width', 100)

print("修正后的原始数据集：")
print(df.to_string(index=True))  # 使用to_string确保格式正确
print(f"\n数据集信息:")
print(f"- 总样本数: {len(df)}")
print(f"- 特征数: {len(df.columns)-1}")
print(f"- 类别分布: 是={sum(df['类别']=='是')}, 否={sum(df['类别']=='否')}")

修正后的原始数据集：
    年龄 有工作 有自己的房子 信贷情况 类别
0   青年     否           否     一般   否
1   青年     否           否       好   否
2   青年     是           否       好   是
3   青年     是           是     一般   是
4   青年     否           否     一般   否
5   中年     否           否     一般   否
6   中年     否           否       好   否
7   中年     是           是       好   是
8   中年     否           是   非常好   是
9   中年     否           是   非常好   是
10  老年     否           是   非常好   是
11  老年     否           是       好   是
12  老年     是           否       好   是
13  老年     是           否   非常好   是
14  老年     否           否     一般   否

数据集信息:
- 总样本数: 15
- 特征数: 4
- 类别分布: 是=9, 否=6


In [16]:
# 计算熵的函数
def calculate_entropy(labels):
    total_count = len(labels)
    if total_count == 0:
        return 0
    
    label_counts = Counter(labels)
    entropy = 0.0
    
    for count in label_counts.values():
        probability = count / total_count
        entropy -= probability * math.log2(probability)
    
    return entropy

# 计算信息增益
def calculate_information_gain(data, feature_name, target_name='类别'):
    total_entropy = calculate_entropy(data[target_name])
    
    feature_values = data[feature_name].unique()
    weighted_entropy = 0.0
    total_count = len(data)
    
    for value in feature_values:
        subset = data[data[feature_name] == value]
        subset_entropy = calculate_entropy(subset[target_name])
        subset_weight = len(subset) / total_count
        weighted_entropy += subset_weight * subset_entropy
    
    information_gain = total_entropy - weighted_entropy
    return information_gain

# 选择最佳特征
def choose_best_feature(data, features, target_name='类别'):
    best_feature = None
    best_gain = -1
    
    print("各特征的信息增益:")
    print("-" * 30)
    for feature in features:
        gain = calculate_information_gain(data, feature, target_name)
        print(f"  {feature:10} : {gain:.4f}")
        
        if gain > best_gain:
            best_gain = gain
            best_feature = feature
    
    return best_feature, best_gain

# 测试信息增益计算
print("\n" + "="*50)
print("信息增益计算")
print("="*50)

initial_entropy = calculate_entropy(df['类别'])
print(f"初始熵: {initial_entropy:.4f}")

features = ['年龄', '有工作', '有自己的房子', '信贷情况']
best_feature, best_gain = choose_best_feature(df, features)
print(f"\n最佳分割特征: '{best_feature}', 信息增益: {best_gain:.4f}")

# 决策树节点类
class TreeNode:
    def __init__(self, feature_name=None, feature_value=None, is_leaf=False, label=None):
        self.feature_name = feature_name
        self.feature_value = feature_value
        self.is_leaf = is_leaf
        self.label = label
        self.children = {}
    
    def add_child(self, feature_value, child_node):
        self.children[feature_value] = child_node
    
    def __repr__(self):
        if self.is_leaf:
            return f"叶节点(类别={self.label})"
        else:
            return f"决策节点(特征={self.feature_name})"

# 构建决策树
def build_decision_tree(data, features, target_name='类别', depth=0, max_depth=10):
    if depth > max_depth:
        majority_label = Counter(data[target_name]).most_common(1)[0][0]
        return TreeNode(is_leaf=True, label=majority_label)
    
    labels = data[target_name].tolist()
    
    # 如果所有样本属于同一类别
    if len(set(labels)) == 1:
        return TreeNode(is_leaf=True, label=labels[0])
    
    # 如果没有特征可用
    if len(features) == 0:
        majority_label = Counter(labels).most_common(1)[0][0]
        return TreeNode(is_leaf=True, label=majority_label)
    
    # 选择最佳特征
    best_feature, best_gain = choose_best_feature(data, features, target_name)
    print(f"{'  '*depth}深度{depth}: 选择特征 '{best_feature}'")
    
    node = TreeNode(feature_name=best_feature)
    remaining_features = [f for f in features if f != best_feature]
    
    # 对每个特征值创建分支
    for feature_value in data[best_feature].unique():
        subset = data[data[best_feature] == feature_value]
        
        if len(subset) == 0:
            majority_label = Counter(labels).most_common(1)[0][0]
            child_node = TreeNode(is_leaf=True, label=majority_label, feature_value=feature_value)
        else:
            child_node = build_decision_tree(subset, remaining_features, target_name, depth+1, max_depth)
            child_node.feature_value = feature_value
        
        node.add_child(feature_value, child_node)
    
    return node

# 构建决策树
print("\n" + "="*50)
print("决策树构建过程")
print("="*50)
decision_tree = build_decision_tree(df, features)

# 打印决策树
def print_tree(node, indent=""):
    if node.is_leaf:
        print(f"{indent}└── 类别: {node.label}")
    else:
        print(f"{indent}{node.feature_name}?")
        children_items = list(node.children.items())
        for i, (feature_value, child_node) in enumerate(children_items):
            connector = "├── " if i < len(children_items) - 1 else "└── "
            print(f"{indent} {connector}{feature_value}", end="")
            if child_node.is_leaf:
                print(f" → 类别: {child_node.label}")
            else:
                print()
                new_indent = indent + (" │   " if i < len(children_items) - 1 else "     ")
                print_tree(child_node, new_indent)

print("\n" + "="*50)
print("最终决策树")
print("="*50)
print_tree(decision_tree)

# 预测函数
def predict(tree, sample):
    if tree.is_leaf:
        return tree.label
    
    feature_value = sample[tree.feature_name]
    
    if feature_value in tree.children:
        return predict(tree.children[feature_value], sample)
    else:
        return None

# 测试预测
print("\n" + "="*50)
print("预测测试")
print("="*50)
test_samples = [
    {'年龄': '青年', '有工作': '否', '有自己的房子': '否', '信贷情况': '一般'},
    {'年龄': '中年', '有工作': '是', '有自己的房子': '是', '信贷情况': '好'},
    {'年龄': '老年', '有工作': '否', '有自己的房子': '是', '信贷情况': '非常好'}
]

print("测试样本预测结果:")
for i, sample in enumerate(test_samples, 1):
    prediction = predict(decision_tree, sample)
    print(f"  样本{i}: {sample}")
    print(f"      预测结果: {prediction}")
    print()

# 评估模型
print("\n" + "="*50)
print("模型评估")
print("="*50)
correct = 0
predictions = []

for i, row in df.iterrows():
    prediction = predict(decision_tree, row)
    actual = row['类别']
    predictions.append(prediction)
    
    if prediction == actual:
        correct += 1

accuracy = correct / len(df) * 100
print(f"训练集准确率: {accuracy:.2f}% ({correct}/{len(df)})")

# 创建详细结果DataFrame并美化显示
results_df = df.copy()
results_df['预测'] = predictions
results_df['是否正确'] = results_df['类别'] == results_df['预测']
results_df['是否正确'] = results_df['是否正确'].map({True: '✓', False: '✗'})

print("\n详细预测结果:")
print(results_df.to_string(index=True))


信息增益计算
初始熵: 0.9710
各特征的信息增益:
------------------------------
  年龄         : 0.0830
  有工作        : 0.3237
  有自己的房子     : 0.4200
  信贷情况       : 0.3630

最佳分割特征: '有自己的房子', 信息增益: 0.4200

决策树构建过程
各特征的信息增益:
------------------------------
  年龄         : 0.0830
  有工作        : 0.3237
  有自己的房子     : 0.4200
  信贷情况       : 0.3630
深度0: 选择特征 '有自己的房子'
各特征的信息增益:
------------------------------
  年龄         : 0.2516
  有工作        : 0.9183
  信贷情况       : 0.4739
  深度1: 选择特征 '有工作'

最终决策树
有自己的房子?
 ├── 否
 │   有工作?
 │    ├── 否 → 类别: 否
 │    └── 是 → 类别: 是
 └── 是 → 类别: 是

预测测试
测试样本预测结果:
  样本1: {'年龄': '青年', '有工作': '否', '有自己的房子': '否', '信贷情况': '一般'}
      预测结果: 否

  样本2: {'年龄': '中年', '有工作': '是', '有自己的房子': '是', '信贷情况': '好'}
      预测结果: 是

  样本3: {'年龄': '老年', '有工作': '否', '有自己的房子': '是', '信贷情况': '非常好'}
      预测结果: 是


模型评估
训练集准确率: 100.00% (15/15)

详细预测结果:
    年龄 有工作 有自己的房子 信贷情况 类别 预测 是否正确
0   青年     否           否     一般   否   否        ✓
1   青年     否           否       好   否   否        ✓
2   青年     是           否       好  